In [1]:
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(tidyr))
suppressPackageStartupMessages(library(tibble))
suppressPackageStartupMessages(library(ggplot2))
suppressPackageStartupMessages(library(stringr))
library(parallel)
library(pheatmap)
library(ggsci)
library(RColorBrewer)
library(viridis)
library(reshape2)
library(ggpubr)
library(tidyverse)

source('/home/jupyter/BRI_Figures_Final_V1/helper_function/helper_function_IHA.r')

Loading required package: viridisLite


Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ forcats   1.0.0     ✔ purrr     1.0.2
✔ lubridate 1.9.3     ✔ readr     2.1.4
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


# BRI

In [2]:
meta_data=read.csv("/home/jupyter/BRI_Figures_Final_V1/Dataset/scRNA_meta_data-2024-05-09.csv")
meta_data_subset<-meta_data  %>% filter (!sample.visitName%in% c('Flu Year 1 Day 7',
                                                              'Flu Year 2 Day 7'))%>% 
filter(Covid_exclusion=='no',sample.daysSinceFirstVisit<600)

In [4]:
aggregated_count_file_list<-paste0("/home/jupyter/BRI_Figures_Final_V1/Dataset/scRNA/BRI/Average_LogNormalized_Expression/Average_LogNormalized_Expression_of_Celltypes_by_Sample_AIFI_L3/",meta_data_subset$pbmc_sample_id,".csv")
df_list<-mclapply(aggregated_count_file_list,function(x){
df<-read.csv(x,check.names = FALSE, row.names = 1)
colnames(df)<-paste0(gsub("^.*/(.*)\\.csv$", "\\1", x),":",colnames(df))
return (df)    
},mc.cores=60)

In [5]:
#selected_genes<-c("CXCR5",'TIGIT','PDCD1','LGALS3','CCR4',"SLAMF1","GATA3","KLRB1", "ITGA4", "TOX", "CD84","CCR8", "IL4R", "NOTCH1")#TH2 activity
selected_genes<-c("ZBTB38")


In [6]:
df<-filter_genes_and_celltype(df_list, selected_genes, c('Adaptive NK cell'), longformat = TRUE)


Warning message in FUN(X[[i]], ...):
“Cell type Adaptive NK cell not found in the dataframe.”


In [7]:
df<-left_join(df,meta_data_subset)

Joining with `by = join_by(pbmc_sample_id)`


In [8]:
df<-df %>% 
mutate(AgeGroup=case_when(cohort.cohortGuid=="BR1"~"Young", TRUE~'Older'))

In [9]:
write.csv(df,'ZBTB38_Addaptive_NK_BRI_OverTime.csv')